

# Download the public datasets

To start downloading and performing analysis on the datasets, please follow the link below.

1)  [Accessing public data with ONE](https://int-brain-lab.github.io/ONE/notebooks/one_quickstart.html)

## Installation
### Environment
To use IBL data you will need a python environment with python > 3.7. To create a new environment from scratch you can install [anaconda](https://www.anaconda.com/products/distribution#download-section) and follow the instructions below to create a new python environment (more information can also be found [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html))

```
conda create --name ibl python=3.9
```
Make sure to always activate this environment before installing or working with the IBL data
```
conda activate ibl
```

### Install packages

To use IBL data you will need to install the ONE-api package. We also recommend installing ibllib. These can be installed via pip.
```python
pip install ONE-api
pip install ibllib
```

### Setting up credentials
Credentials can be setup in a python terminal in the following way

In [ ]:
from one.api import ONE
pw = 'international'
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password=pw, silent=True)

## Getting Started

### Exploring data
To get a feel for the structure of the data we recommended first downloading the alf data for a single repeated site session and exploring how the data is stored locally on disk. An example alf folder can be downloaded from [here](https://ibl-brain-wide-map-public.s3.amazonaws.com/sample_data/mainenlab/Subjects/ZM_2241/2020-01-30/001/alf_ZM2241_2020-01-30_001.zip) or alternatively, we can use the ONE-api to search for sessions released as part of this paper and download the data this way.

The sessions that contain data released as part of the reproducible ephys data paper can be found in the following way

In [ ]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

# Find sessions that have data and are tagged for the repeated site paper
rep_site_sessions = one.alyx.rest('sessions', 'list', dataset_types='spikes.times', tag='2022_Q2_IBL_et_al_RepeatedSite')
# Take the first session
example_sess = rep_site_sessions[0]
# Each session has a unique experiment id
eid = example_sess['id']

We can download all files in the **alf** collection

In [ ]:
# Download all data in alf collection
files = one.load_collection(eid, 'alf', download_only=True)

# Show where files have been downloaded to
print(f'Files downloaded to {files[0].parent}')

To download the spikesorting data we need to find out which probe label (probeXX above) was used for this session. This can be done by finding the probe insertion associated with this session

In [ ]:
insertion = one.alyx.rest('insertions', 'list', session=eid)[0]
probe_label = insertion['name']
files = one.load_collection(eid, f'alf/{probe_label}/pykilosort', download_only=True)

# Show where files have been downloaded to
print(f'Files downloaded to {files[0].parent}')

To load in the data we can use some of the following loading methods

In [ ]:
# Load in all trials datasets
trials = one.load_object(eid, 'trials', collection='alf')

# Load in a single wheel dataset
wheel_times = one.load_dataset(eid, '_ibl_wheel.timestamps.npy')

### Loading different objects
Examples for loading different objects can be found in the following tutorials [here](https://int-brain-lab.github.io/iblenv/loading_examples.html)

### More information on ONE
To get a better understading of the ONE-api and the various methods available we recommend working through these tutorials

* [ONE quickstart](https://int-brain-lab.github.io/iblenv/notebooks_external/one_quickstart.html)
* [Searching with ONE](https://int-brain-lab.github.io/ONE/notebooks/one_search/one_search.html)
* [Listing with ONE](https://int-brain-lab.github.io/ONE/notebooks/one_list/one_list.html)
* [Loading with ONE](https://int-brain-lab.github.io/ONE/notebooks/one_load/one_load.html)
* [ONE alyx rest API](https://int-brain-lab.github.io/ONE/notebooks/one_advanced/one_advanced.html)(advanced)

## Advanced examples
### Example 1: Searching for sessions from a specific lab

If you want to use data associated to a given lab only, you could simply query for the whole dataset as shown above,
and filter `rep_site_sessions` for the key "lab" of a given value, for example:

In [ ]:
lab_name = 'mrsicflogellab'
sessions_lab = [item for item in rep_site_sessions if item['lab'] == lab_name]

However, if you wanted to query only the data for a given lab, it might be most judicious to first
know the list of all labs available, select an arbitrary lab name from it, and query the specific sessions from it.

To get this list, use [one.alyx.rest](https://openalyx.internationalbrainlab.org/docs/#labs-list)

In [ ]:
# List of labs (and all metadata information associated)
labs = one.alyx.rest('labs', 'list',
                     django='session__data_dataset_session_related__tags__name,2022_Q2_IBL_et_al_RepeatedSite')
# Note the change in the django filter compared to searching over 'sessions'

# Example lab name
lab_name = labs[0]['name']  # e.g. 'mrsicflogellab'

# Searching for RS sessions with specific lab name
sessions_lab = one.alyx.rest('sessions', 'list', dataset_types='spikes.times', lab=lab_name,
                             tag='2022_Q2_IBL_et_al_RepeatedSite')

## Information and troubleshooting
- Issues with the data? Post an issue here: <https://github.com/int-brain-lab/iblenv/issues>
- General questions about the dataset or paper  Email: <info@internationalbrainlab.org>